<a href="https://colab.research.google.com/github/jrivera15/TallerIADG/blob/main/practicas/Pr%C3%A1ctica3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>🌍 Introducción al análisis de datos geoespaciales con QGIS y Python</center>

### <center>**Práctica 3:**</center>

### <center>Crear puntos, líneas, polígonos en Python</center>  
<center>
María Janneth Rivera Reyna<br>
Universidad de Sonora<br>
AI-Linkup 2025, Octubre 2025<br>
</center>  
<br>

Objetivo:
Crear datos vectoriales digitales dibujando geometrías en Python y agregando atributos (Digitalizar), familiarizarse con algunas librerías de manejo de datos geoespaciales y explorar algunas de sus herramientas. Finalmente, visualizar los datos y exportar a archivos que contengan estos datos geoespaciales.



## Instalación de librerías

Geopandas tiene varias dependecias a otras librerías, tales como:
pandas, numpy, shapely, entre otras.

Nota: Si quieres correr local y usas un gestor de librerías como Anaconda/Miniconda puedes instalar con:
<code>conda install geopandas</code>

In [ ]:
!pip install geopandas mapclassify

## Importando librerías

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import matplotlib.pyplot as plt

## Creando geometrías: puntos, líneas, polígonos

Una forma de crear geometrías con <code>Shapely</code>, es definiéndolas a través de coordenadas geográficas, como lo son la *latitud y longitud*.<br><br>

Puedes utilizar herramientas como **Google Maps** o **Google Earth** para ayudarte a obtener las coordenadas de lugares en específico que te sirvan para crear las geometrías.<br><br>


###<center>Point(x, y) --> donde x=longitud, y=latitud</center><br><br>

Esto es porque <code>Shapely</code> sigue la convención cartesiana (x, y), donde la longitud (este-oeste) corresponde al eje de las X, y la latitud (norte-sur) corresponde al eje de las Y. <code>Geopandas</code> y muchos otros formatos GIS utilizan esta convención para mantener compatibilidad entre sistemas.

#### Creando un punto

In [ ]:
# Crear un punto: Unison Rectoría
punto = Point(-110.96030979779613, 29.083597383593112)

#### Creando una línea

In [ ]:
# Crear una línea: ruta Rectoría -> Salida Colosio
linea = LineString([(-110.96096839209038, 29.083392391885077),
                    (-110.96090523921285, 29.08291933254036),
                    (-110.96205101284805, 29.08273799254891),
                    (-110.9616630737432, 29.080498811565406)])

#### Creando un polígono

In [ ]:
# Crear un polígono: área verde Emiliana de Zubeldía
poligono = Polygon([(-110.95975208843697, 29.084144286407092),
                    (-110.95954725475629, 29.083066932612304),
                    (-110.9587696454129, 29.083143176482118),
                    (-110.95890240798371, 29.083892352375223)])

#### Creando GeoPandas Dataframe

In [ ]:
# Guardarlos en un GeoDataFrame
gdf = gpd.GeoDataFrame({'tipo': ['punto','linea','poligono'],
                        'nombre': ['Rectoría','Salida Colosio','Emiliana de Zubeldía'],
                        'geometry':[punto,linea,poligono]},
                        crs="EPSG:4326")  # WGS84 geográfico
gdf

### Exportando archivos

Los **SHP** files aceptan sólo un tipo de geometría, y representan una única capa vectorial, por lo tanto para este ejercicio se deberán guardar en archivos separados.

**GeoPackage** es un formato más moderno, el cual soporta múltiples capas dentro del mismo archivo y cada capa puede tener un tipo de geometría diferente.

In [ ]:
# Exportar a SHP files (separados por tipo de geometría)
gdf[gdf.geometry.type == 'Point'].to_file("puntos_unison.shp")
gdf[gdf.geometry.type == 'LineString'].to_file("lineas_unison.shp")
gdf[gdf.geometry.type == 'Polygon'].to_file("poligonos_unison.shp")

# Exportar a CSV
gdf.to_csv("geometrias_python.csv", index=False)

# Exportar a Geopackage
gdf.to_file("geometrias_python.gpkg", layer='geometrias', driver="GPKG")


## Explorando algunas funcionalidades

<code>Geopandas</code> puede crear mapas interactivos basados en <code>Folium</code>: una librería basada en Javascript para crear mapas interactivos a partir de datos geoespaciales.

El método <code>explore()</code> nos regresa un objeto <code>folium.Map</code>, que nos permite rápidamente visualizar el mapa:

In [ ]:
gdf.explore()

### Centroides y límites
<code>Geopandas</code> trae incorporadas varios atributos y métodos que nos permiten realizar algunos cálculos rápidamente:

In [ ]:
# Obtener centroides
gdf['centroide'] = gdf.centroid

# Obtener límites
gdf['limites'] = gdf.boundary

gdf


Vamos a graficar los centroides y límites:

In [ ]:
# Graficar con matplotlib
ax = gdf["geometry"].plot(figsize=(10, 6), edgecolor="black") # límites
gdf["centroide"].plot(ax=ax, color="red", markersize=20)      # centroides
plt.title("Centroides y límites de las geometrías")
plt.show()

### Distancia y área
Otra funcionalidad interesante es la de calcular distancias y áreas.

Para las distancias, vamos a definir un punto de referencia, en este caso *Rectoría*:

In [ ]:
rectoria = gdf['geometry'][0]

Ahora, vamos a calcular el área de cada geometría en el GDF así como la distancia al punto de referencia (Rectoría).


In [ ]:
gdf.area

In [ ]:
gdf.distance(rectoria)

🔴 Regresa y observa los **warnings**.

Estos resultados están dados en grados, ya que el sistema de referencia de coordenadas que definimos cuando se creó el GDF es uno geográfico, por lo tanto no nos sirve para realizar mediciones como distancias, áreas, etc.

Vamos a reproyectar, utilizando un sistema de referencia de coordenadas proyectado:

In [ ]:
# Reproyectamos y guardamos en un nuevo GDF
gdf_metros = gdf.to_crs(epsg=32612)  # WGS 84 / Zona UTM 12N

In [ ]:
# Redefinimos el punto de referencia, ahora con un CRS en metros
rectoria_metros = gdf_metros['geometry'][0]

# Calculamos la distancia
gdf_metros['dist_rectoria_m'] = gdf_metros.distance(rectoria_metros)

# Calculamos el área
gdf_metros['area_m2'] = gdf_metros.area

gdf_metros

In [ ]:
gdf_metros.explore()

## Tarea 3

1.	Crea 3 puntos dentro de la ciudad de Hermosillo para ubicar escuelas y agrega campos como nombre(Texto), nivel(Texto – primaria, secundaria, etc).
2.	Crea una línea para identificar una ruta para bicis. Agrega campos como nombre, long_m (usa el método length() para calcularla).
3.	Crea 3 polígonos para identificar parques. Agrega campos como nombre, uso, calcula el area_m2 y la distancia a Rectoría (o algún otro punto de referencia de tu interés).
4. Exportar como SHP y GeoPackage.
